# Environment Variables

In [1]:
%env ENABLE_WEIGHTSANDBIAS=false
%env ENABLE_MLFLOW=false
%env ENABLE_KG_SOURCE=true
%env ONTOLOGY_DATABASE=Ontology_database_agent_test #note this is my local vectordatabase name, we need this because ENABLE_KG_SOURCE=true
%env WEAVIATE_API_KEY="1#FG!NsmUVEs" #note this is my local vectordatabase key
%env OLLAMA_API_ENDPOINT=http://host.docker.internal:11434 #note this is my local vectordatabase name
%env OLLAMA_MODEL=nomic-embed-text #note this is my local vectordatabase name

env: ENABLE_WEIGHTSANDBIAS=false
env: ENABLE_MLFLOW=false
env: ENABLE_KG_SOURCE=false
env: ONTOLOGY_DATABASE=Ontology_database_agent_test #note this is my local vectordatabase name, we need this because ENABLE_KG_SOURCE=true
env: WEAVIATE_API_KEY="XFG!NQmUVEC&8" #note this is my local vectordatabase key
env: OLLAMA_API_ENDPOINT=http://host.docker.internal:11434 #note this is my local vectordatabase name
env: OLLAMA_MODEL=nomic-embed-text #note this is my local vectordatabase name


# Configuration Files

In [15]:
import json

In [4]:
agent_config = {
    "extractor_agent": {
        "role": "Neuroscience Named Entity Recognition (NER) Extractor Agent",
        "goal": "Perform Named Entity Recognition (NER) on neuroscience {literature} and return structured JSON output.",
        "backstory": (
            "You are an AI assistant specialized in processing neuroscience and who do not hallucinate. "
            "Your expertise includes recognizing and categorizing named entities such as anatomical regions, "
            "experimental conditions, and cell types. Your responses strictly adhere to JSON format, ensuring "
            "accurate and structured data extraction for downstream applications."
        ),
        "llm": {
            "model": "openrouter/openai/gpt-4o-2024-11-20",
            "base_url": "https://openrouter.ai/api/v1",
            "frequency_penalty": 0.1,
            "temperature": 0.7,
            "seed": 53,
            "api_key": "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"
        }
    },
    "alignment_agent": {
        "role": "Neuroscience Named Entity Recognition (NER) Concept Alignment Agent",
        "goal": "Perform concept alignment to the extracted Named Entity Recognition (NER) by extractor_agent {extracted_structured_information} and return structured JSON output.",
        "backstory": (
            "You are an AI assistant specialized in processing neuroscience concept alignment with structured models, "
            "i.e., ontologies or schemas and who do not hallucinate. Your expertise includes recognizing and categorizing "
            "extracted named entities such as anatomical regions, experimental conditions, and cell types and aligning the "
            "recognized named entities such as cell types with corresponding ontological terms. Your responses strictly "
            "adhere to JSON format, ensuring accurate and structured data extraction for downstream applications."
        ),
        "llm": {
            "model": "openrouter/openai/gpt-4o-2024-11-20",
            "base_url": "https://openrouter.ai/api/v1",
            "frequency_penalty": 0.1,
            "temperature": 0.7,
            "seed": 53,
            "api_key": "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"
        }
    },
    "judge_agent": {
        "role": "Neuroscience Named Entity Recognition (NER) Judge Agent",
        "goal": "Evaluate the {aligned_structured_information} based on predefined criteria and generate a structured JSON output reflecting the assessment results.",
        "backstory": (
            "You are an AI assistant with expert knowledge in neuroscience and structured models, i.e., ontologies or schemas, "
            "and someone who does not hallucinate. Your task is to evaluate the {aligned_structured_information} based on the "
            "accuracy and quality of the alignment. Assign the score between 0-1 with 1 being the highest score of your evaluation. "
            "Your responses strictly adhere to JSON format, ensuring accurate and structured data extraction for downstream applications."
        ),
        "llm": {
            "model": "openrouter/openai/gpt-4o-2024-11-20",
            "base_url": "https://openrouter.ai/api/v1",
            "frequency_penalty": 0.1,
            "temperature": 0.7,
            "seed": 53,
            "api_key": "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"
        }
    }
}

task_config = {
    "extraction_task": {
        "description": (
            "From the given literature extract named entities from neuroscience statements. "
            "A named entity is anything that can be referred to with a proper name. "
            "Some common named entities in neuroscience articles are animal species (e.g., mouse, drosophila, zebrafish), "
            "anatomical regions (e.g., neocortex, mushroom body, cerebellum), experimental conditions (e.g., control, tetrodotoxin treatment, Scn1a knockout), "
            "and cell types (e.g., pyramidal neuron, direction-sensitive mechanoreceptor, oligodendrocyte)\n\n"
            "Literature:\n{literature}"
        ),
        "expected_output": (
            'output format: json\n'
            'Example output:\n'
            '"extracted_terms": {\n'
            '  "1": [\n'
            '    {\n'
            '      "entity": "mouse",\n'
            '      "label": "ANIMAL_SPECIES",\n'
            '      "sentence": "These particles were visualized by fluorescent immunohistochemistry using mouse monoclonal anti-human myelin basic protein (MBPh) antibody (clone SMI-99).",\n'
            '      "start": 79,\n'
            '      "end": 84,\n'
            '      "paper_location": "methods",\n'
            '      "paper_title": "Concentration of myelin debris-like myelin basic protein-immunoreactive particles in the distal (anterior)-most part of the myelinated region in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ],\n'
            '  "2": [\n'
            '    {\n'
            '      "entity": "oligodendrocyte",\n'
            '      "label": "CELL_TYPE",\n'
            '      "sentence": "Individual oligodendrocytes provide, on average, 16 near axons with single myelin segments about 200 µm in length (Butt and Ransom, 1993).",\n'
            '      "start": 14,\n'
            '      "end": 29,\n'
            '      "paper_location": "discussion",\n'
            '      "paper_title": "Concentration of myelin debris-like myelin basic protein-immunoreactive particles in the distal (anterior)-most part of the myelinated region in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ],\n'
            '  "3": [\n'
            '    {\n'
            '      "entity": "cerebellum",\n'
            '      "label": "ANATOMICAL_REGION",\n'
            '      "sentence": "Myelin basic protein, human (MBPh) ... on immunoblots of the mouse cerebellum (Dyer et al.).",\n'
            '      "start": 293,\n'
            '      "end": 303,\n'
            '      "paper_location": "methods",\n'
            '      "paper_title": "Concentration of myelin debris-like myelin basic protein-immunoreactive particles in the distal (anterior)-most part of the myelinated region in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ]\n'
            '}'
        ),
        "agent_id": "extractor_agent"
    },
    "alignment_task": {
        "description": (
            "Take the output of extractor_agent {extracted_structured_information} as input and perform the concept alignment using the ontological concepts. "
            "A concept alignment is anything where you align the given entity to the matching concept aka class from a ontology or a schema."
        ),
        "expected_output": (
            'output format: json\n'
            'Example output:\n'
            '"aligned_ner_terms": {\n'
            '  "1": [\n'
            '    {\n'
            '      "entity": "oligodendrocyte",\n'
            '      "label": "CELL_TYPE",\n'
            '      "ontology_id": "CL:0000128",\n'
            '      "ontology_label": "Oligodendrocyte",\n'
            '      "sentence": "Individual oligodendrocytes provide, on average, 16 near axons with single myelin segments about 200 µm in length (Butt and Ransom, 1993).",\n'
            '      "start": 14,\n'
            '      "end": 29,\n'
            '      "paper_location": "discussion",\n'
            '      "paper_title": "Concentration of myelin debris-like myelin basic protein-immunoreactive particles in the distal (anterior)-most part of the myelinated region in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ],\n'
            '  "2": [\n'
            '    {\n'
            '      "entity": "cerebellum",\n'
            '      "label": "ANATOMICAL_REGION",\n'
            '      "ontology_id": "UBERON:0002037",\n'
            '      "ontology_label": "Cerebellum",\n'
            '      "sentence": "Myelin basic protein, human (MBPh) ... on immunoblots of the mouse cerebellum (Dyer et al.).",\n'
            '      "start": 293,\n'
            '      "end": 303,\n'
            '      "paper_location": "methods",\n'
            '      "paper_title": "Concentration of myelin debris-like ... in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ]\n'
            '}'
        ),
        "agent_id": "alignment_agent"
    },
    "judge_task": {
        "description": (
            "Take the output of alignment agent {aligned_structured_information} as input and perform the following evaluation:\n"
            "1. Assess the quality and accuracy of the alignment with the ontology or schema in {aligned_structured_information}.\n"
            "2. Assign a score between 0 and 1 as a judge_score.\n"
            "3. Update the {aligned_structured_information} adding the judge_score."
        ),
        "expected_output": (
            'output format: json\n'
            'Example output:\n'
            '"judge_ner_terms": {\n'
            '  "1": [\n'
            '    {\n'
            '      "entity": "oligodendrocyte",\n'
            '      "label": "CELL_TYPE",\n'
            '      "ontology_id": "CL:0000128",\n'
            '      "ontology_label": "Oligodendrocyte",\n'
            '      "sentence": "Individual oligodendrocytes provide, on average, 16 near axons with single myelin segments about 200 µm in length (Butt and Ransom, 1993).",\n'
            '      "start": 14,\n'
            '      "end": 29,\n'
            '      "judge_score": "0.8",\n'
            '      "paper_location": "discussion",\n'
            '      "paper_title": "Concentration of myelin debris-like ... in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ],\n'
            '  "2": [\n'
            '    {\n'
            '      "entity": "cerebellum",\n'
            '      "label": "ANATOMICAL_REGION",\n'
            '      "ontology_id": "UBERON:0002037",\n'
            '      "ontology_label": "Cerebellum",\n'
            '      "sentence": "Myelin basic protein, human (MBPh) ... on immunoblots of the mouse cerebellum (Dyer et al.).",\n'
            '      "start": 293,\n'
            '      "end": 303,\n'
            '      "judge_score": "0.54",\n'
            '      "paper_location": "methods",\n'
            '      "paper_title": "Concentration of myelin debris-like ... in the normal rat optic nerve",\n'
            '      "doi": "10.1101/2025.03.19.643597"\n'
            '    }\n'
            '  ]\n'
            '}'
        ),
        "agent_id": "judge_agent"
    }
}



#search key to be used in vector database
search_key  = {
    "search_key": [
        "entity",
        "label"
    ]
}

#embedding models
embedderconfig = {
    "embedder_config": {
        "provider": "ollama",
        "config": {
            "api_base": "http://localhost:11434",
            "model": "nomic-embed-text:latest"
        }
    }
}

sourcestr = "Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function."

In [5]:

import nest_asyncio
import asyncio

nest_asyncio.apply()

In [6]:
from structsense import kickoff

result = kickoff(
      agentconfig=agent_config,
      taskconfig=task_config,
      embedderconfig=embedderconfig, 
        input_source=sourcestr,
    knowledgeconfig=search_key
)

2025-04-08 16:57:34,331 - utils.utils - INFO - Trying paths: ['Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.', '/Users/tekrajchhetri/Documents/brainypedia_codes_design/crew_ner_framework/structsense/example/programmatic_access/Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.', '/Users/tekrajchhetri/Documents/brainypedia_codes_design/crew_ner_framework/structsense/example/programmatic_access/Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.', '/Users/tekrajchhetri/Documents/brainypedia_codes_design/crew_ner_framework/structsense/example/programmatic_access/Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.']
2025-04-08 16:57:34,333 - utils.utils - I

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: StructSenseFlow                                                                                          │
│  ID: ed3d98e0-5d20-4d25-817b-ed8203dc4a37                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-08 16:57:34,354 - structsense.app - INFO - Initializing StructSenseFlow
2025-04-08 16:57:34,390 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-04-08 16:57:34,738 - structsense.app - INFO - Memory systems initialized successfully


🌊 Flow: StructSenseFlow
    ID: ed3d98e0-5d20-4d25-817b-ed8203dc4a37
└── 🧠 Starting Flow...

 Flow started with ID: ed3d98e0-5d20-4d25-817b-ed8203dc4a37
2025-04-08 16:57:34,741 - crewai.flow.flow - INFO - Flow started with ID: ed3d98e0-5d20-4d25-817b-ed8203dc4a37


...

2025-04-08 16:58:32,075 - structsense.app - INFO - Judgment complete with 0 judged terms


🌊 Flow: StructSenseFlow
    ID: ed3d98e0-5d20-4d25-817b-ed8203dc4a37
├── Flow Method Step
├── ✅ Completed: process_inputs
├── ✅ Completed: extracted_structured_information
├── ✅ Completed: align_structured_information
└── ✅ Completed: judge_alignment

✅ Flow Finished: StructSenseFlow
├── Flow Method Step
├── ✅ Completed: process_inputs
├── ✅ Completed: extracted_structured_information
├── ✅ Completed: align_structured_information
└── ✅ Completed: judge_alignment

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: StructSenseFlow                                                                                          │
│  ID: ed3d98e0-5d20-4d25-817b-ed8203dc4a37                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-04-08 16:58:32,088 - structsense.app - INFO - Flow completed successfully


In [12]:
print(type(result))

print(result)

<class 'dict'>
{'judged_structured_information': {'1': [{'entity': 'APOE', 'label': 'GENE', 'ontology_id': 'HGNC:613', 'ontology_label': 'APOE', 'sentence': 'Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.', 'start': 26, 'end': 30, 'judge_score': 1.0, 'paper_location': 'unknown', 'paper_title': 'unknown', 'doi': 'unknown'}], '2': [{'entity': 'neurodegenerative disorders', 'label': 'DISORDER', 'ontology_id': 'MONDO:0021060', 'ontology_label': 'neurodegenerative disease', 'sentence': 'Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.', 'start': 49, 'end': 74, 'judge_score': 1.0, 'paper_location': 'unknown', 'paper_title': 'unknown', 'doi': 'unknown'}], '3': [{'entity': 'astrocytes', 'label': 'CELL_TYPE', 'ontology_id': 'CL:0000127', 'ontology_label': 'astrocyte', 'sentence': 'Additionally, mutations in the APOE gene ha

In [14]:
import json
print(json.dumps(result, indent=2))

{
  "judged_structured_information": {
    "1": [
      {
        "entity": "APOE",
        "label": "GENE",
        "ontology_id": "HGNC:613",
        "ontology_label": "APOE",
        "sentence": "Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.",
        "start": 26,
        "end": 30,
        "judge_score": 1.0,
        "paper_location": "unknown",
        "paper_title": "unknown",
        "doi": "unknown"
      }
    ],
    "2": [
      {
        "entity": "neurodegenerative disorders",
        "label": "DISORDER",
        "ontology_id": "MONDO:0021060",
        "ontology_label": "neurodegenerative disease",
        "sentence": "Additionally, mutations in the APOE gene have been linked to neurodegenerative disorders, impacting astrocytes and microglia function.",
        "start": 49,
        "end": 74,
        "judge_score": 1.0,
        "paper_location": "unknown",
        "paper_title": "unknow